# Parameters

In [1]:
lead_time = 21 # days
goal_wos = 5
item = 5370

# First Half

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load the data from the provided Excel file
file_path = '/content/dp_tester_1.csv'
data = pd.read_csv(file_path)

# getting unique sku's
unique_sku = data['Vendor Stk Nbr'].unique()
#data = data[data['Future Valid'] == 1]
future_week = pd.to_datetime(data['WM Week End Date'][0]) + pd.Timedelta(days=lead_time)

In [3]:
needed_inv_frames = []

# Loop over unique SKU
for sku in unique_sku:
    # Filter data for the current SKU
    data_filtered_sku = data[data['Vendor Stk Nbr'] == sku].reset_index(drop=True)
    valid_stores = data_filtered_sku[data_filtered_sku['Valid Stores'] == 1]
    unique_store = valid_stores['Store Nbr'].unique()
    # Loop over unique stores for the current SKU
    for store in unique_store:
        # Filter data for the current store and SKU
        data_filtered = data_filtered_sku[data_filtered_sku['Store Nbr'] == store].reset_index(drop=True)

        # Lists to store the calculated data
        ending_inventory = []
        weeks_of_supply = []
        needed_inventory = []

        # Initialize the inventory starting from the pipeline quantity if the DataFrame is not empty
        if not data_filtered.empty:
            current_inventory = data_filtered['Pipeline Qty'].iloc[0]

            for index in range(len(data_filtered)):
                # Subtract this week's forecast from the current inventory
                current_inventory -= data_filtered['Forecast Units'][index]
                ending_inventory.append(current_inventory)

                # Calculate the weeks of supply based on the next week's forecast, if available
                if index < len(data_filtered) - 1:
                    next_week_forecast = data_filtered['Forecast Units'][index + 1]
                    weeks_supply = current_inventory / next_week_forecast if next_week_forecast else 0
                else:
                    weeks_supply = 0  # No forecast available for the last week
                weeks_of_supply.append(weeks_supply)

                # Calculate needed inventory if weeks of supply is less than 6
                if weeks_supply < goal_wos and index >= 0:  # Start calculating needed inventory from week 4
                    additional_required = (goal_wos * next_week_forecast) - current_inventory
                    additional_required = np.round(max(additional_required, 0))  # Round to the nearest whole number
                    additional_required = additional_required / data_filtered['Vnpk Qty'][index] # additional need in vendor packs
                    additional_required = np.ceil(additional_required) # rounding up the nearest integer
                    additional_required = additional_required * data_filtered['Vnpk Qty'][index] # converting back into eaches
                else:
                    additional_required = 0
                needed_inventory.append(additional_required)

                # Adjust the current inventory for the next iteration
                current_inventory += additional_required

            # Create new columns in the DataFrame
            data_filtered['Ending Week On Hand'] = ending_inventory
            data_filtered['WOS'] = weeks_of_supply
            data_filtered['Needed'] = needed_inventory
            data_filtered['Order Date'] = pd.to_datetime(data_filtered['WM Week End Date']) - pd.Timedelta(days=lead_time)
            data_filtered['Month'] = data_filtered['Order Date'].dt.month
            data_filtered['Year'] = data_filtered['Order Date'].dt.year
            data_filtered['Week'] = data_filtered['Order Date'].dt.isocalendar().week

            # Append the filtered data for the current store and SKU to the needed_inv DataFrame
            needed_inv_frames.append(data_filtered)

needed_inv = pd.concat(needed_inv_frames, ignore_index=True)

# Reset index of the resulting DataFrame
needed_inv.reset_index(drop=True, inplace=True)

In [4]:
# this makes the first 3 weeks zero
# mask = pd.to_datetime(needed_inv['WM Week End Date']) < future_week
# needed_inv.loc[mask, 'Needed'] = 0

In [5]:
needed_inv.head()

,WM Week End Date,Prime Item Description,Prime Item Number,Vendor Stk Nbr,Store Nbr,Store Name,Valid Stores,Pipeline Qty,Forecast Units,Vnpk Qty,Ending Week On Hand,WOS,Needed,Order Date,Month,Year,Week
0,2024-06-07 00:00:00.0,BLKST 2PK SPATULA,575326563,5229,2420,"BELLE VERNON, PA",1.0,10.0,1.40,4,8.60,5.243902,0.0,2024-05-17,5,2024,20
1,2024-06-14 00:00:00.0,BLKST 2PK SPATULA,575326563,5229,2420,"BELLE VERNON, PA",NaN,0.0,1.64,4,6.96,4.936170,0.0,2024-05-24,5,2024,21
2,2024-06-21 00:00:00.0,BLKST 2PK SPATULA,575326563,5229,2420,"BELLE VERNON, PA",NaN,0.0,1.41,4,5.55,27.750000,0.0,2024-05-31,5,2024,22
3,2024-06-28 00:00:00.0,BLKST 2PK SPATULA,575326563,5229,2420,"BELLE VERNON, PA",NaN,0.0,0.20,4,5.35,6.948052,0.0,2024-06-07,6,2024,23
4,2024-07-05 00:00:00.0,BLKST 2PK SPATULA,575326563,5229,2420,"BELLE VERNON, PA",NaN,0.0,0.77,4,4.58,9.956522,0.0,2024-06-14,6,2024,24


In [6]:
sum_needed = needed_inv.groupby(['Vendor Stk Nbr', 'Week','Year'])['Needed'].sum().reset_index()
sum_needed

,Vendor Stk Nbr,Week,Year,Needed
0,2193,1,2025,0.0
1,2193,3,2025,0.0
2,2193,4,2025,0.0
3,2193,5,2025,0.0
4,2193,6,2025,0.0
...,...,...,...,...
1757,5331,48,2024,774.0
1758,5331,49,2024,18.0
1759,5331,50,2024,0.0
1760,5331,51,2024,0.0


In [7]:
# Pivot the DataFrame
pivot_data = sum_needed.pivot(index='Vendor Stk Nbr', columns=['Week','Year'], values='Needed')

# Reset index to make 'Vendor Stk Nbr' a regular column
pivot_data.reset_index(inplace=True)

# Rename the columns to include 'week' prefix
pivot_data.columns.name = None  # Remove the name of the columns index
pivot_data.columns = ['Vendor Stk Nbr'] + [f'Week_{week}' for week in pivot_data.columns[1:]]

# Display the pivoted DataFrame
print(pivot_data)

    Vendor Stk Nbr  Week_(1, 2025)  Week_(3, 2025)  Week_(4, 2025)  \
0             2193             0.0             0.0             0.0   
1             2194             0.0             0.0             0.0   
2             2260             2.0             0.0             3.0   
3             4100          3120.0          2898.0          3738.0   
4             4104          2970.0          3870.0          2382.0   
5             4106          4104.0          4092.0          4206.0   
6             4107          3168.0          3492.0          4380.0   
7             4109          1956.0          2112.0          2508.0   
8             4114          3060.0          9588.0          8136.0   
9             4121          2982.0          4542.0          4260.0   
10            4122          1602.0          2784.0          2316.0   
11            4123         27132.0         33360.0          8928.0   
12            4138          1245.0          2310.0          2650.0   
13            4172  

# Second Half

In [8]:
import pandas as pd
import numpy as np

# Load the data from the provided Excel file
file_path = '/content/dp_tester_2.csv'
data = pd.read_csv(file_path)

# getting unique sku's
unique_sku = data['Vendor Stk Nbr'].unique()
#data = data[data['Future Valid'] == 1]
data.tail()

,WM Week End Date,Prime Item Description,Prime Item Number,Vendor Stk Nbr,Store Nbr,Store Name,Valid Stores,Pipeline Qty,Forecast Units,Vnpk Qty
4008169,2025-06-06 00:00:00.0,GRIDDLE PRESS,665051939,5924,147,DENISON TX,NaN,0.0,0.38,6
4008170,2025-06-06 00:00:00.0,GRIDDLE PRESS,665051939,5924,301,GADSDEN AL,NaN,0.0,0.14,6
4008171,2025-06-06 00:00:00.0,GRIDDLE PRESS,665051939,5924,253,"JERSEYVILLE, IL.",NaN,0.0,0.31,6
4008172,2025-06-06 00:00:00.0,GRIDDLE PRESS,665051939,5924,29,JEFFERSON CITY MO,NaN,0.0,0.04,6
4008173,2025-06-06 00:00:00.0,GRIDDLE PRESS,665051939,5924,45,JONESBORO AR,NaN,0.0,0.01,6


In [9]:
needed_inv_frames = []

# Loop over unique SKU
for sku in unique_sku:
    # Filter data for the current SKU
    data_filtered_sku = data[data['Vendor Stk Nbr'] == sku].reset_index(drop=True)
    valid_stores = data_filtered_sku[data_filtered_sku['Valid Stores'] == 1]
    unique_store = valid_stores['Store Nbr'].unique()
    # Loop over unique stores for the current SKU
    for store in unique_store:
        # Filter data for the current store and SKU
        data_filtered = data_filtered_sku[data_filtered_sku['Store Nbr'] == store].reset_index(drop=True)

        # Lists to store the calculated data
        ending_inventory = []
        weeks_of_supply = []
        needed_inventory = []

        # Initialize the inventory starting from the pipeline quantity if the DataFrame is not empty
        if not data_filtered.empty:
            current_inventory = data_filtered['Pipeline Qty'].iloc[0]

            for index in range(len(data_filtered)):
                # Subtract this week's forecast from the current inventory
                current_inventory -= data_filtered['Forecast Units'][index]
                ending_inventory.append(current_inventory)

                # Calculate the weeks of supply based on the next week's forecast, if available
                if index < len(data_filtered) - 1:
                    next_week_forecast = data_filtered['Forecast Units'][index + 1]
                    weeks_supply = current_inventory / next_week_forecast if next_week_forecast else 0
                else:
                    weeks_supply = 0  # No forecast available for the last week
                weeks_of_supply.append(weeks_supply)

                # Calculate needed inventory if weeks of supply is less than 6
                if weeks_supply < goal_wos and index >= 0:  # Start calculating needed inventory from week 4
                    additional_required = (goal_wos * next_week_forecast) - current_inventory
                    additional_required = np.round(max(additional_required, 0))  # Round to the nearest whole number
                    additional_required = additional_required / data_filtered['Vnpk Qty'][index] # additional need in vendor packs
                    additional_required = np.ceil(additional_required) # rounding up the nearest integer
                    additional_required = additional_required * data_filtered['Vnpk Qty'][index] # converting back into eaches
                else:
                    additional_required = 0
                needed_inventory.append(additional_required)

                # Adjust the current inventory for the next iteration
                current_inventory += additional_required

            # Create new columns in the DataFrame
            data_filtered['Ending Week On Hand'] = ending_inventory
            data_filtered['WOS'] = weeks_of_supply
            data_filtered['Needed'] = needed_inventory
            data_filtered['Order Date'] = pd.to_datetime(data_filtered['WM Week End Date']) - pd.Timedelta(days=lead_time)
            data_filtered['Month'] = data_filtered['Order Date'].dt.month
            data_filtered['Year'] = data_filtered['Order Date'].dt.year
            data_filtered['Week'] = data_filtered['Order Date'].dt.isocalendar().week

            # Append the filtered data for the current store and SKU to the needed_inv DataFrame
            needed_inv_frames.append(data_filtered)

needed_inv = pd.concat(needed_inv_frames, ignore_index=True)

# Reset index of the resulting DataFrame
needed_inv.reset_index(drop=True, inplace=True)

In [ ]:
# this makes the first 3 weeks zero
# mask = pd.to_datetime(needed_inv['WM Week End Date']) < future_week
# needed_inv.loc[mask, 'Needed'] = 0

In [ ]:
needed_inv.head()

In [ ]:
sum_needed = needed_inv.groupby(['Vendor Stk Nbr', 'Week','Year'])['Needed'].sum().reset_index()
sum_needed

In [ ]:
# Pivot the DataFrame
pivot_data_2 = sum_needed.pivot(index='Vendor Stk Nbr', columns=['Week','Year'], values='Needed')

# Reset index to make 'Vendor Stk Nbr' a regular column
pivot_data_2.reset_index(inplace=True)

# Rename the columns to include 'Month' prefix
pivot_data_2.columns.name = None  # Remove the name of the columns index
pivot_data_2.columns = ['Vendor Stk Nbr'] + [f'Week_{week}' for week in pivot_data_2.columns[1:]]

# Display the pivoted DataFrame
print(pivot_data_2)

In [ ]:
# needed_inv.to_excel('needed_inv.xlsx')
# pivot_data.to_excel('/content/fcst_report_2.xlsx')

# Combining both

In [ ]:
combined_df = pd.concat([pivot_data, pivot_data_2], axis=0)
combined_df.to_excel('/content/fcst_report.xlsx')

# Weekly Breakdown by Item by Store

In [16]:
conditioner = needed_inv[needed_inv['Vendor Stk Nbr'] == item]

conditioner.to_excel('needed_inv.xlsx')
# pivot_data.to_excel('/content/fcst_report_1.xlsx')